# Машинное обучение

## Загрузим обработанные данные обзоров и соответсвующих обзору оценок

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('Clean_goods.csv')
df=df.dropna().reset_index(drop=True)
df.head()

,Review,Sentiment
0,служба доставка оставаться крайне негативный в...,0
1,недобросовестный транспортный компания,0
2,хорошо почта россия,0
3,ужасный контора,0
4,большой срок доставка нарушение срок посылка у...,0


In [3]:
len(df)

7281

## Будем работать с list of tuple обзоров-тонов

In [4]:
print df['Review'].iloc[:][0]

служба доставка оставаться крайне негативный впечатление


In [5]:
clean_reviews=zip(df['Review'].iloc[:],df['Sentiment'].iloc[:])

## Посмотрим сколько положительных и отрицательных отзывов в нашей выборке

In [6]:
def BadGoodReview(Reviews_Sentiments):
    bad=0
    good=0
    for item in Reviews_Sentiments:
        if int(item[1])==1:
            good=good+1
        elif int(item[1])==0:
            bad=bad+1
    return (good,bad)

In [7]:
print 'good','bad'
print BadGoodReview(clean_reviews)

good bad
(3076, 4205)


## сделаем выборку 50 на 50 для удобства

In [8]:
def fifty(reviews_sentiments):
    result=[]
    num_of_min=min(BadGoodReview(reviews_sentiments))
    for review in clean_reviews:
        if int(review[1])==1:
            result.append((review[0],review[1]))
        elif int(review[1])==0 and num_of_min>0:
            result.append((review[0],review[1]))
            num_of_min=num_of_min-1
    return result

In [9]:
clean_reviews=fifty(clean_reviews)

In [10]:
print 'good','bad'
print BadGoodReview(clean_reviews)

good bad
(3076, 3076)


# Обучим модели на разных классификаторах, с функциями взвешивания tf-idf и count vectorizer, с разными комбинациями n-грамм

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit, cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
import numpy as np

# массив n-граммных схем, которые будут использоваться в работе
# например, (1, 3) означает униграммы + биграммы + триграммы
ngram_schemes = [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6)]

for ngram_scheme in ngram_schemes:

    print('N-gram Scheme:', ngram_scheme)

    count_vectorizer = CountVectorizer(analyzer = "word",ngram_range=ngram_scheme,tokenizer = None, preprocessor = None, stop_words = None,max_features = 3000)
    tfidf_vectorizer = TfidfVectorizer(analyzer = "word", ngram_range=ngram_scheme,max_features=3000)

    vectorizers = [count_vectorizer, tfidf_vectorizer]
    vectorizers_names = ['Count Vectorizer', 'TF-IDF Vectorizer']

    for i in range(len(vectorizers)):
        print(vectorizers_names[i])
        vectorizer = vectorizers[i]

        X = vectorizer.fit_transform([sentence[0] for sentence in clean_reviews])
        y = [int(sentiment[1]) for sentiment in clean_reviews]
        np.asarray(X)
        
        cv = ShuffleSplit( n_splits=5, test_size=0.2, random_state=0)
        
        classifiers=[]
        classifiers.append(('RF', RandomForestClassifier(n_estimators = 100)))
        classifiers.append(('AB', AdaBoostClassifier()))
        classifiers.append(('DT', DecisionTreeClassifier()))
        classifiers.append(('LR', LogisticRegression(solver = 'lbfgs', max_iter = 500, multi_class = 'auto')))
        classifiers.append(('NB', GaussianNB()))
        classifiers.append(('SVR', SVC(gamma = 2, C = 1.0))) # gamma - коэффициент ядра для 'rbf' - radial basis function, 'poly' and 'sigmoid'
        classifiers.append(('MLP', MLPClassifier(alpha = 0.01, max_iter = 200, solver = 'lbfgs', tol = 0.001)))
        classifiers.append(('KNN', KNeighborsClassifier(2)))
        
        for clf in classifiers:
            print clf[0], ':'
            print cross_val_score(clf[1],X.toarray(),y,cv=cv).mean()

('N-gram Scheme:', (1, 1))
Count Vectorizer


NameError: name 'np' is not defined

# Как видно лучший результат показывают униграммы + tf-idf мера+ SVR классификатор. Будем работать с ним

In [12]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics 

## перемешаем обзоры

In [13]:
import random
random.shuffle(clean_reviews)

In [14]:
train,test=np.split(clean_reviews, [int(.8*len(clean_reviews))])

In [15]:
ngram_scheme = (1, 1)
vectorizer = TfidfVectorizer(analyzer = "word", ngram_range=ngram_scheme,max_features=3000)

In [16]:
train_data_features = vectorizer.fit_transform([sentence[0] for sentence in train])

In [17]:
np.asarray(train_data_features)

array(<4921x3000 sparse matrix of type '<type 'numpy.float64'>'
	with 17874 stored elements in Compressed Sparse Row format>, dtype=object)

## тестовая

In [18]:
test_data_features = vectorizer.transform([sentence[0]for sentence in test])
np.asarray(test_data_features)

array(<1231x3000 sparse matrix of type '<type 'numpy.float64'>'
	with 4126 stored elements in Compressed Sparse Row format>, dtype=object)

In [46]:
%%time
clf= SVC(gamma = 1.0, C = 1.5, probability=True)
clf=clf.fit(train_data_features, [int(sentiment[1]) for sentiment in train])


result= clf.predict(test_data_features)

print'accuracy', metrics.accuracy_score([int(sentiment[1]) for sentiment in test],result)
print 'precision', metrics.precision_score([int(sentiment[1]) for sentiment in test],result)
print'recall', metrics.recall_score([int(sentiment[1]) for sentiment in test],result)
print'f1 measure', metrics.f1_score([int(sentiment[1]) for sentiment in test],result)

accuracy 0.8870836718115354
precision 0.8869426751592356
recall 0.8912
f1 measure 0.8890662410215483
Wall time: 6.34 s


In [47]:
print clf.predict_proba(test_data_features[0])
print clf.predict(test_data_features[0])

[[0.96209551 0.03790449]]
[0]


## Остальные классификаторы были отброшены из -за меньшей точности, несмотря на то, что некоторые из них (например LR) обучаются заметно быстрее

## Сохраним классификатор и векторизатор

In [48]:
import cPickle
# save the classifier
with open('my_dumped_classifier.pkl', 'wb') as fid:
    cPickle.dump(clf, fid) 

In [49]:
with open('vectorizer.pkl', 'wb') as fid:
    cPickle.dump(vectorizer,fid)

In [50]:
df=pd.DataFrame()
df['Review']=[i[0] for i in test]
df['Sentiment']=[int(i[1]) for i in test]
df.to_csv('testRev.csv',index=False)

In [55]:
print df['Review'][1]

ужасно
